In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, Model, load_model
from keras.callbacks import Callback, TensorBoard
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Activation, BatchNormalization, Dropout, Input
from keras.optimizers import SGD, RMSprop, Adadelta, Adagrad, Adam
from keras.applications import vgg16
from keras.regularizers import l2
import cv2
from keras.preprocessing.image import ImageDataGenerator
import collections

Using TensorFlow backend.


In [2]:
def preprocess(x):
    mean = np.mean(x, axis=(1, 2, 3), keepdims=True)
    x -= mean
    return x


def mirror(x):
    mirror_x = x[..., ::-1, :]
    return np.vstack([x, mirror_x])


# def to_rgb(x):
#     x = x[..., 0]
#     rgb_shape = x.shape + (3,)
#     x_rgb = np.zeros(rgb_shape)
#     x_rgb[...] = x[..., np.newaxis]
#     return x_rgb

train_X, train_y, val_X, val_y, test_X, test_y = np.load("data.npy")
train_X, val_X = preprocess(train_X), preprocess(val_X)
train_X = mirror(train_X)
train_y = np.vstack([train_y, train_y])

In [3]:
def get_model():
    model = Sequential()
#     model.add(Convolution2D(3, 3, 3, border_mode='same', input_shape=(48, 48, 1)))
#     model.add(Activation('relu'))

    model.add(Convolution2D(64, 3, 3, border_mode='same', input_shape=(48, 48, 1)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    model.add(Convolution2D(64, 3, 3, border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(Convolution2D(128, 3, 3, border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    model.add(Convolution2D(128, 3, 3, border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(Convolution2D(256, 3, 3, border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    model.add(Convolution2D(256, 3, 3, border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    model.add(Convolution2D(256, 3, 3, border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(Convolution2D(512, 3, 3, border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    model.add(Convolution2D(512, 3, 3, border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    model.add(Convolution2D(512, 3, 3, border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(1024))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(7, activation='softmax'))

    model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics=["categorical_accuracy"])

    return model

model = get_model()
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 48, 48, 64)    640         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 48, 48, 64)    256         convolution2d_1[0][0]            
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 48, 48, 64)    0           batchnormalization_1[0][0]       
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 48, 48, 64)    0           activation_1[0][0]               
___________________________________________________________________________________________

In [4]:
class History(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.acc = []
        self.val_losses = []
        self.val_acc = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.acc.append(logs.get('categorical_accuracy'))

    def on_epoch_end(self, epoch, logs={}):
        self.val_losses.append(logs.get('val_loss'))
        self.val_acc.append(logs.get('val_categorical_accuracy'))

history = History()
dic = collections.defaultdict(list)

In [11]:
for i in range(30):
    model.fit(train_X, train_y, batch_size=128, nb_epoch=1, validation_data=(val_X, val_y), callbacks=[history, TensorBoard()])
    dic['loss'].extend(history.losses)
    dic['acc'].extend(history.acc)
    dic['val_loss'].extend(history.val_losses)
    dic['val_acc'].extend(history.val_acc)
    model.save("vgg_models/vgg" + str(i))

np.save('vgg_history/history', dic)

Train on 57418 samples, validate on 3589 samples
Epoch 1/1
57418/57418 [==============================] - 230s - loss: 1.9010 - categorical_accuracy: 0.2808 - val_loss: 1.5389 - val_categorical_accuracy: 0.4132